## 6.1 处理文本数据
### 6.1.1 单词和字符的one-hot编码
**代码清单 6-1** 单词级的one-hot编码（简单示例）

In [ ]:
import numpy as np

# 初始数据：每个样本是列表的一个元素（本例中的样本是一个句子，但也可以是一整篇文档）
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# 构建数据中所有标记的索引
token_index = {}
for sample in samples:
    # 利用split方法对样本进行分词。在实际应用中，还需要从样本中去掉标点和特殊字符
    for word in sample.split():
        if word not in token_index:
            # 为每个唯一单词指定一个唯一索引。注意，没有为索引编号0指定单词
            token_index[word] = len(token_index) + 1

# 对样本进行分词。只考虑每个样本前max_length个单词
max_length = 10
# 将结果存在results中
results = np.zeros(shape=(len(samples), max_length, max(token_index.values()) + 1))

for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.

In [ ]:
token_index

In [ ]:
results

**代码清单 6-2** 字符级的one-hot编码（简单示例）

In [ ]:
import numpy as np
import string

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
# 所有可打印的ASCII字符
characters = string.printable
token_index = dict(zip(characters, range(1, len(characters) + 1)))

max_length = 50
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, character in enumerate(sample[:max_length]):
        index = token_index.get(character)
        results[i, j, index] = 1.

In [ ]:
token_index.keys()

In [ ]:
results.shape

**代码清单 6-3** 用Keras实现单词级的one-hot编码

In [ ]:
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# 创建一个分词器(tokenizer)，设置为只考虑前1000个最常见的单词
tokenizer = Tokenizer(num_words=1000)
# 构建单词索引
tokenizer.fit_on_texts(samples)

# 将字符串转换为整数索引组成的列表
sequences = tokenizer.texts_to_sequences(samples)

# 也可以直接得到one-hot二进制表示。这个分词器也支持除one-hot编码外的其它向量化模式
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')

# 找回单词索引
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
sequences

In [ ]:
one_hot_results.shape

In [ ]:
word_index

**代码清单 6-4** 使用散列表技巧的单词级的one-hot编码（简单示例）

In [ ]:
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# 将单词保存为长度为1000的向量，如果单词数量接近1000个（或者更多）
# 那么会遇到很多散列冲突，这会降低这种编码方法的准确性
dimensionality = 1000
max_length = 10

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        # 将单词散列为0~1000范围内的一个随机整数索引
        index = abs(hash(word)) % dimensionality
        results[i, j, index] = 1.

In [ ]:
results

### 6.1.2 使用词嵌入
**1. 利用Embedding层学习词嵌入**

**代码清单 6-5** 将一个Embedding层实例化

In [ ]:
from keras.layers import Embedding

# Embedding层至少需要两个参数：
# 标记的个数（这里是1000，即最大单词索引+1）和嵌入的维度（这里是64）
embedding_layer = Embedding(1000, 64)

**代码清单 6-6** 加载IMDB数据，准备用于Embedding层

In [ ]:
from keras.datasets import imdb
from keras import preprocessing

# 作为特征的单词个数
max_features = 10000
# 在这么多单词后截断文本（这些单词都属于前max_features个最常见的单词）
maxlen = 20

# 将数据加载为整数列表
(x_train, y_train), (x_test, y_test) = imdb.load_data(
    num_words=max_features)

# 将整数列表转换成形状为(samples, maxlen)的二维整数张量
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

**代码清单 6-7** 在IMDB数据上使用Embedding层和分类器

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()
# 指定Embedding层的最大输入长度，以便后面将嵌入输入展平。
# Embedding层激活的形状为(samples, maxlen, 8)
model.add(Embedding(10000, 8, input_length=maxlen))

# 将三维的嵌入张量展平成形状为(samples, maxlen * 8)的二维张量
model.add(Flatten())

# 在上面添加分类器
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

In [ ]:
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)